In [ ]:
# Imports from __future__ in case we're running Python 2
from __future__ import division, print_function
from __future__ import absolute_import, unicode_literals

# Our numerical workhorses
import numpy as np
import scipy.integrate

# Import pyplot for plotting
import matplotlib.pyplot as plt

# Seaborn, useful for graphics
import seaborn as sns

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.mpl
import bokeh.plotting

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables SVG graphics inline.  There is a bug, so uncomment if it works.
# %config InlineBackend.figure_formats = {'svg',}

# This enables high resolution PNGs. SVG is preferred, but has problems
# rendering vertical and horizontal lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# Financial Algebra #

### My refresher notes on Algebra commonly used in Finance ###

\begin{align}
(AB)^{-1} = B^{-1}A^{-1}
\end{align}

A matrix that has an inverse is called *Non Singular Matrix*, otherwise its called 
*Singular Matrix*. A singular matrix has at least one row (or column) that has a linear 
relationship with another row (or column).


### Determinant of a matrix ###

$$det(A) = |A| = det
 \begin{bmatrix}
  a & b \\
  c & d \\  
 \end{bmatrix} = \mathbf{ad} - \mathbf{bc} $$
Determinant of a matrix is zero if the matrix is singular. A matrix with zero determinant has no inverse. To inverse a matrix, use determinant and cofactors as below
$$\mathbf{A}^{-1} = \begin{bmatrix}
  a & b \\
  c & d \\  
 \end{bmatrix}^{-1} = \frac{1}{\mid A \mid} = \frac{1}{\mathbf{ad} - \mathbf{bc}} \begin{bmatrix}
  d & -b \\
  -c & a \\  
 \end{bmatrix}$$

In [ ]:
A = np.array[[1,2],[3,4]]
det = np.linalg.det(A)
print det
def matrix_cofactor(matrix):
    C = np.zeros(matrix.shape)
    nrows, ncols = C.shape
    for row in xrange(nrows):
        for col in xrange(ncols):
            minor = matrix[np.array(range(row)+range(row+1,nrows))[:,np.newaxis],
                           np.array(range(col)+range(col+1,ncols))]
            C[row, col] = (-1)**(row+col) * np.linalg.det(minor)
    return C
cofactorMatrix = matrix_cofactor(np.asmatrix(A))
prnt cofactorMatrix
a_inverse = np.multiply(det, cofactorMatrix)
print (a_inverse)